In [37]:
import numpy as np
import torch
# Set print options for PyTorch
# edgeitems: number of array items in summary at beginning and end of each dimension
# threshold: total number of array items printed
# linewidth: number of characters per line for printing
torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

In [38]:
bikes_numpy = np.loadtxt(
    "../data/p1ch4/bike-sharing-dataset/hour-fixed.csv", 
    dtype=np.float32, 
    delimiter=",", 
    skiprows=1, 
    converters={1: lambda x: float(x[8:10])}) # <1>
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [39]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [40]:
17520 / 24 / 365

2.0

In [41]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [42]:
# We transpose to have N x C x L = 730 x 17 x 24
# where N is the number of days, C the number of channels, and L the length of each sequence of measurements
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [43]:
daily_bikes[:1].shape

torch.Size([1, 17, 24])

In [44]:
first_day = bikes[:24].long()
first_day.shape

torch.Size([24, 17])

In [45]:
weather_onehot = torch.zeros(first_day.shape[0], 4)


In [46]:
first_day[:,9].unsqueeze(1) - 1 # so it's 0 to 3

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [1],
        [1],
        [1],
        [1]])

In [47]:
weather_onehot.scatter_(
    dim=1, 
    index=first_day[:,9].unsqueeze(1).long() - 1, # <1>
    value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [48]:
weather_onehot.shape, bikes[:24, :9].shape, first_day[:,10:].shape

(torch.Size([24, 4]), torch.Size([24, 9]), torch.Size([24, 7]))

In [49]:
torch.cat((bikes[:24, :9], bikes[:24, 10:], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          6.0000,  0.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000,
         13.0000, 16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [50]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
                                   daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [51]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape, daily_bikes.shape

(torch.Size([730, 4, 24]), torch.Size([730, 17, 24]))

In [52]:
daily_bikes = torch.cat((daily_bikes[:,:9,:], daily_bikes[:,10:,:], daily_weather_onehot), dim=1)

In [53]:
daily_bikes.shape

torch.Size([730, 20, 24])

In [54]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [55]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [56]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
                         / torch.std(temp))